In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

!pip install efficientnet -q
!pip install iterative-stratification -q
!pip install tensorflow_addons -q
!pip install focal-loss
import numpy as np
import pandas
import tensorflow as tf
import os
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold,MultilabelStratifiedShuffleSplit
from focal_loss import BinaryFocalLoss

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
def onehot(image,label):
    CLASSES = 19
    return image,tf.one_hot(label,CLASSES)
def cutmix(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMSIZE[IMS]
    CLASSES = 19
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        lab1 = tf.cast(lab1,tf.float32)
        lab2 = tf.cast(lab2,tf.float32)
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2


def mixup(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = IMSIZE[IMS]
    CLASSES = 19
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        lab1 = tf.cast(lab1,tf.float32)
        lab2 = tf.cast(lab2,tf.float32)
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2


def transform(image,label):
    # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
    DIM = IMSIZE[IMS]
    CLASSES = 19
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    image3, label3 = mixup(image, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image4,label4



In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

DIM =256
n_class = 19
def _parse_image_function(example_proto,augment = True):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string)
    }
    single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.reshape( tf.io.decode_raw(single_example['image'],out_type=np.dtype('uint8')), (DIM,DIM, 3))
    mask =  tf.reshape(tf.io.decode_raw(single_example['label'],out_type=np.dtype('uint8')),[n_class])
    image = tf.dtypes.cast(image, tf.float32)
    mask = tf.dtypes.cast(mask, tf.float32)
    image = image/255.
    if augment: # https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu

        if tf.random.uniform(()) > 0.5:
            image = tf.image.flip_left_right(image)
            mask = tf.image.flip_left_right(mask)

        if tf.random.uniform(()) > 0.4:
            image = tf.image.flip_up_down(image)
            mask = tf.image.flip_up_down(mask)

        if tf.random.uniform(()) > 0.5:
            image = tf.image.rot90(image, k=1)
            mask = tf.image.rot90(mask, k=1)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_saturation(image, 0.7, 1.3)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_contrast(image, 0.8, 1.2)
    
    return tf.cast(image, tf.float32),tf.cast(mask, tf.float32)

def load_dataset(filenames, ordered=False, augment = False):
    AUTO = tf.data.experimental.AUTOTUNE
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda ex: _parse_image_function(ex, augment = augment), num_parallel_calls=AUTO)
    return dataset






def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True,augment_mixup_cutmix=False
                  , repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(True)
    
    AUTO = tf.data.experimental.AUTOTUNE
    #slices = paths if labels is None else (paths, labels)
    dset = load_dataset(paths)
    #dset = tf.data.Dataset.from_tensor_slices(slices)
    #dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    #dset = dset.map(transform, num_parallel_calls=AUTO) if augment else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize)
    dset = dset.map(transform,num_parallel_calls=AUTO) if augment_mixup_cutmix else dset
    dset = dset.prefetch(AUTO)
    
    return dset

In [ ]:
def count_data_items(filenames):
    img_num = 0
    for i in filenames:
        img_num +=int(i.split('_')[-2])
    return img_num

In [ ]:
train_list=[]
valid_list=[]

In [ ]:
gs_csv = pd.read_csv('../input/hpagscsv/gs.csv')
val_index=[0] #  decide which dataset to be the valid one
for i in gs_csv.index:
    temp=gs_csv.loc[i,'URL']
    if i not in val_index:
        train_list.extend(tf.io.gfile.glob(i+'/*'))
    else:
        valid_list.extend(tf.io.gfile.glob(i+'/*'))

        
#'gs://kds-d2130e02868047c380c97c6ae5d31a4180f74d21ceefbf5d4c9e531e' 0
#'gs://kds-53777409f1b5814b19e2bc8b542d4af1163b9ac03623463085b45210' hpatrain-tfrec-1
#'gs://kds-37f6a0e35a9bb77b47960da902f2fad44df121760c080ff2a9a04257' hpatrain-tfrec-2
#'gs://kds-e3c610c144b38135540fa8471f82968eab5f181530e876ba09d20fad' hpatrain-tfrec-3        
#'gs://kds-004df2f4910e2bf06fc13b08ef1bdb77467be005cff1fea662d07c73' hpaexset-02tfrec
#'gs://kds-16cffa22de689b2549ff0bb18319210db5cc4566db005e35282ba293' hpaexset-3-5tfrec
#'gs://kds-5774c5e377adbfc07a2cde16dba0ef50cf79f4459280fd7ee4bb8ff3' hpaexset-6-8tfrec
#'gs://kds-20636028bcd9f63dc66fd5790cc609e05e2f836911243ef50159b008' hpaexset-9-11
#'gs://kds-c90ca8f6f770bac1662c25b73f8cd83eb94897a0632df7a20f19dbec' hpaexset-12-14tfrec
#'gs://kds-b0bf359a1d74d177f49a1f189b11142ccbaaacd7a6eb4d108f1d4fca' hpaexset-15-17tfrec
#'gs://kds-1b20087af52f7720ba8347a73cca5be42a92d20c0092489ef7b0af05' hpaexset-18-20tfrec
#'gs://kds-79a5c17813bfefd7dc4239f7ae438cd4483b713fd0610221321aa69a' hpaexset-21-23tfrec
#'gs://kds-ca83f53b04345ee5b510e901a2711465d54e134c7b7bd7d9ca52b54a' hpaexset-24-26tfrec
#'gs://kds-8ffb8be72389a3683a6925456e96ad5b7bc6516fc95457f7e2a16691' hpaexset-27-30tfrec
#'gs://kds-d33bfe4ffc561ba62405827097415975c2a0ef7babcd9f693f7aa643' hpaexset-31-34tfrec
#'gs://kds-1afb9a2e010e75655e2e686dafaf1a9d1ac2feeaf01737e8a9506d35' hpaexset-35-37tfrec

In [ ]:
IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 256)
IMS = 8

strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 32
AUG_BATCH = BATCH_SIZE




train_dataset = build_dataset(
    train_list,  bsize=BATCH_SIZE,cache=False,augment_mixup_cutmix=True
)

valid_dataset = build_dataset(
    valid_list,  bsize=BATCH_SIZE,
    cache=False,repeat=False, shuffle=False, augment=False
)


In [ ]:
# define the model

try:
    n_labels = 19#train_labels.shape[1]
except:
    n_labels = 1
    
with strategy.scope():
    '''
    model = tf.keras.models.load_model(
            '/content/gdrive/MyDrive/Kaggle/model_RGB_0.0051.h5'
        )
    '''
    
    model = tf.keras.Sequential([
        efn.EfficientNetB0(
            input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
            weights='imagenet',
            include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(n_labels, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',#BinaryFocalLoss(gamma=5),#'binary_crossentropy',#tfa.losses.SigmoidFocalCrossEntropy(),
        metrics=[tf.keras.metrics.AUC(multi_label=True)])
        
    model.summary()

In [ ]:
colour = '_RGB'
#steps_per_epoch = len(train_paths) // BATCH_SIZE
steps_per_epoch = count_data_items(train_list) // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model_RGB_{val_loss:.4f}.h5", save_best_only=False, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=2,
    verbose=1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv(f'history{colour}.csv')